In [1]:
import numpy as np
from scipy.spatial.transform import Rotation

from numpy.typing import ArrayLike
from typing import Union

In [2]:
def print_rot(r: Rotation):
    roll, pitch, yaw = r.as_euler('xyz', degrees=True)
    print(f"yaw: {yaw:.2f}, pitch: {pitch:.2f}, roll: {roll:.2f}")

In [44]:
class Transform(object):
    pass

class Frame(object):
    """Global frame is one with origin (0,0,0) and unity rotation. Local frame
    is anything else."""
    
    # Choose right vector (arbitrary but stable)
    # use world +Y as up, so right = up × forward
    UP = np.asarray([0, 0, 1])

    def __init__(self, origin: ArrayLike, orientation: Union[ArrayLike, Rotation]):
        self.origin = np.asarray(origin)
        
        if isinstance(orientation, Rotation):
            self.rotation = orientation
        else:
            self.rotation = Frame.to_rotation(orientation)

    def __repr__(self) -> str:
        r, p, y = self.rotation.as_euler('xyz', degrees=True)
        return f"Frame(origin={self.origin.tolist()}, rotation=({y:.2f}, {p:.2f}, {r:.2f}))"

    def from_local_direction(self, local_direction: ArrayLike) -> 'Frame':
        """Frame of the direction relative to this frame"""
        local_rotation = Frame.to_rotation(local_direction)
        roll, pitch, yaw = local_rotation.as_euler('xyz', degrees=True)

        m = self.rotation.as_matrix()
        print(m)
        print(m[:, 1], pitch)
        p = Rotation.from_rotvec(m[:, 1] * pitch, degrees=True)
        print(m[:, 2], yaw)
        y = Rotation.from_rotvec(m[:, 2] * yaw, degrees=True)

        r = y * p * self.rotation
        return Frame(self.origin, r)
    
    def vec_in_frame(self, vec: ArrayLike) -> np.ndarray:
        """Express absolute vector in frame"""
        vec = np.asarray(vec)
        assert vec.shape[-1] == 3

        vec = self.rotation.apply(vec, inverse=True)
        return vec - self.origin

    def cov_in_frame(self, cov: ArrayLike) -> np.ndarray:
        """Express absolute covariance matrix in frame"""
        r = self.rotation.inv().as_matrix()
        
        cov = np.asarray(cov)
        if cov.shape == (3, 3):
            return r @ cov @ r.T
        elif cov.shape == (6, 6):
            ret = np.zeros_like(cov)
            ret[:3, :3] = r @ cov[:3, :3] @ r.T
            ret[3:, :3] = r @ cov[3:, :3] @ r.T
            ret[:3, 3:] = r @ cov[:3, 3:] @ r.T
            ret[3:, 3:] = r @ cov[3:, 3:] @ r.T
            return ret
        else:
            raise Exception(f"Unsupported covariance shape {cov.shape}")

    #def transfrom_
    
    @staticmethod
    def to_rotation(vec: ArrayLike) -> Rotation:
        """
        Derives a quaternion for a coordinate system: 
        Forward=[1,0,0], Left=[0,1,0], Up=[0,0,1].
        The result will have zero roll relative to the XY plane.
        """
        vec = np.asarray(vec)
        
        # 1. Normalize the target forward vector (Local X)
        forward = vec / np.linalg.norm(vec)
        
        # 2. Derive the Local Left vector (Local Y)
        # Crossing Global Up with Forward gives a vector on the XY plane
        left = np.cross(Frame.UP, forward)
        
        # Handle singularity: if target is straight up/down, cross product is zero
        if np.linalg.norm(left) < 1e-6:
            # Default to global Left [0, 1, 0] to resolve ambiguity
            left = np.array([0, 1, 0])
        else:
            left /= np.linalg.norm(left)
        
        # 3. Derive the Re-orthogonalized Local Up vector (Local Z)
        up = np.cross(forward, left)
        
        # 4. Construct Rotation Matrix
        # Columns correspond to Local X (Forward), Local Y (Left), and Local Z (Up)
        matrix = np.column_stack((forward, left, up))
    
        # 5. Convert to Quaternion [w, x, y, z]
        return Rotation.from_matrix(matrix)

class Point3D:
    # measurement coordinates are (0,0,range)
    range: float    
    frame: Frame

    x_stdev: float
    y_stdev: float
    z_stdev: float
    
    def __init__(self, vec: ArrayLike, forward=[1, 0, 0]):
        vec = np.asarray(vec)
        rng = np.linalg.norm(vec)
        self.range = float(rng)
        self.frame = Frame([0, 0, 0], vec)

        self.x_stdev = self.y_stdev = self.z_stdev = 1

    def __repr__(self) -> str:
        return f"Point3D(range={self.range:.2f}, frame={self.frame})"

In [45]:
# identity
f = Frame([0, 0, 0], [1, 0, 0])
assert np.allclose(f.vec_in_frame([1, 0, 0]), [1, 0, 0], atol=.01)

# -45 deg az
f = Frame([0, 0, 0], [1, 1, 0])
assert np.allclose(f.vec_in_frame([1, 0, 0]), [.7, -.7, 0], atol=.01)

# -90 deg az
f = Frame([0, 0, 0], [0, 1, 0])
assert np.allclose(f.vec_in_frame([1, 0, 0]), [0, -1, 0], atol=.01)

# identity
f = Frame([0, 0, 0], [1, 1, 0])
assert np.allclose(f.vec_in_frame([1, 1, 0]), [1.41, 0, 0], atol=.01)

# 45 deg down
f = Frame([0, 0, 0], [1, 1, 1.4142])
assert np.allclose(f.vec_in_frame([1, 1, 0]), [1, 0, -1], atol=.01)

In [51]:
f = Frame([0, 0, 0], [1.732, 1, 0])

# 30 degrees left
e = f.from_local_direction([1.732, 1, 0]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, 0, 60], atol=.01), f"{e}"

# 30 degrees up
e = f.from_local_direction([1.732, 0, 1]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -30, 30], atol=.01), f"{e}"

# 30 degrees up, 30 degrees left
e = f.from_local_direction([1.732, 1, 2/np.sqrt(3)]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -30, 60], atol=.01), f"{e}"

# 30 deg up
f = Frame([0, 0, 0], [1.732, 0, 1])

# 30 degrees up
e = f.from_local_direction([1.732, 0, 1]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -60, 0], atol=.01), f"{e}"

# 30 degrees up, 30 degrees left
e = f.from_local_direction([1.732, 1, 2/np.sqrt(3)]).rotation.as_euler('xyz', degrees=True)
assert np.allclose(e, [0, -30, 60], atol=.01), f"{e}"

[[ 0.86601905 -0.500011    0.        ]
 [ 0.500011    0.86601905  0.        ]
 [ 0.          0.          1.        ]]
[-0.500011    0.86601905  0.        ] 0.0
[0. 0. 1.] 30.000727780827372
[[ 0.86601905 -0.500011    0.        ]
 [ 0.500011    0.86601905  0.        ]
 [ 0.          0.          1.        ]]
[-0.500011    0.86601905  0.        ] -30.00072778082736
[0. 0. 1.] 0.0
[[ 0.86601905 -0.500011    0.        ]
 [ 0.500011    0.86601905  0.        ]
 [ 0.          0.          1.        ]]
[-0.500011    0.86601905  0.        ] -30.00054583061685
[0. 0. 1.] 30.000727780827372
[[ 0.86601905 -0.         -0.500011  ]
 [ 0.          1.         -0.        ]
 [ 0.500011    0.          0.86601905]]
[-0.  1.  0.] -30.00072778082736
[-0.500011   -0.          0.86601905] 0.0
[[ 0.86601905 -0.         -0.500011  ]
 [ 0.          1.         -0.        ]
 [ 0.500011    0.          0.86601905]]
[-0.  1.  0.] -30.00054583061685
[-0.500011   -0.          0.86601905] 30.000727780827372


AssertionError: [-25.10986211 -53.90293928  47.30592393]

In [7]:
p = Point3D([1, 1, 1])
p

Point3D(range=1.73, frame=Frame(origin=[0, 0, 0], rotation=(45.00, -35.26, 0.00)))

## Track in 3D

In [146]:
class Tracker3D:
    """Position + velocity"""
    
    def __init__(self):
        self.x     = np.zeros((6, 1))
        self.P     = np.eye(6)
        self.frame = Frame([0, 0, 0], [1, 0, 0])

        self.H     = np.asarray([[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])
        self.v = None
        self.S = None
        self.K = None

    def initialize(self, x: ArrayLike):
        x = np.asarray(x).flatten()
        self.x[:len(x), 0] = x
    
    def predict(self, dt: float):
        pass
    
    def update(self, meas: Point3D):
        # 1. express measurement in the track frame
        z = np.asarray([meas.range, 0, 0])
        z = self.frame.vec_in_frame(z).reshape((3, 1)) # column vector

        R = np.diag([meas.x_stdev, meas.y_stdev, meas.z_stdev])
        R = self.frame.cov_in_frame(R)
        
        # 2. calculate innovation
        self.S = self.H @ self.P @ self.H.T + R
        self.v = z - self.H @ self.x

        # 3. Kalman gain
        self.K = self.P @ self.H.T @ np.linalg.inv(self.S)

        # 4. filtered mean + cov
        self.x = self.x + self.K @ self.v
        self.P = self.P - self.K @ self.S @ self.K.T

        # 5. frame centered at the new target position
        new_frame = self.frame.from_local_direction(self.x[:3, 0])

        # 6. delta rotation, delta frame
        delta_rot   = new_frame.rotation * self.frame.rotation.inv()
        delta_frame = Frame(np.zeros(3), delta_rot)

        # 7. rotate state to the new frame
        self.P  = delta_frame.cov_in_frame(self.P)

        # rotate position and velocity separately
        self.x[:3, 0] = delta_frame.vec_in_frame(self.x[:3, 0])
        self.x[3:, 0] = delta_frame.vec_in_frame(self.x[3:, 0])

In [152]:
t = Tracker3D()
t.initialize([10, 0, 0])

t.update(Point3D([10, 10, 0]))

In [153]:
t.x

array([[12.07106781],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [154]:
print_rot(t.frame.rotation)

yaw: 0.00, pitch: 0.00, roll: 0.00


In [157]:
t.v

array([[4.14213562],
       [0.        ],
       [0.        ]])

In [156]:
t.K

array([[0.5, 0. , 0. ],
       [0. , 0.5, 0. ],
       [0. , 0. , 0.5],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [21]:
r1 = Rotation.from_euler('xyz', [0, 30, 20], degrees=True)
print_rot(r1)

yaw: 20.00, pitch: 30.00, roll: -0.00


In [27]:
r2 = Rotation.from_euler('xyz', [0, 31, 21], degrees=True)
print_rot(r2)

yaw: 21.00, pitch: 31.00, roll: 0.00


In [28]:
print_rot(r2 * r1.inv())

yaw: 1.00, pitch: 0.94, roll: -0.34
